# Traditional Machine Learning Model

Use SVM, Random forest to predict PVC

Note: This code have to be run after preprocessing.ipynb

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from biosppy import storage
from biosppy.signals import ecg


In [5]:
trainset = pickle.load(open("./trainset.p",'rb'))
testset = pickle.load(open("./testset.p",'rb'))

In [6]:
trainset.head()

,R_interval_mean,R_interval_sd,age,annotation,gender,id,lead,p2p_mean,p2p_std,peak,strip,stripPVC,p2p_diff_form_norm
0,223.000000,34.254562,64,symbol 18 + 111 L 343 ...,0,109,V1,256.708652,11.87702,"[94, 326, 586, 790, 1029, 1269, 1503, 1748, 19...",0 0 0.871194 1 0.871562 2...,False,33.708652
1,221.400000,6.839103,64,symbol 3813 L 4041 L 4264 ...,0,109,V1,256.708652,11.87702,"[3795, 4023, 4249, 4464, 4673, 4890, 5110, 534...",0 3600 -1.317180 3601 -1.265853 3...,False,35.308652
2,233.071429,8.293408,64,symbol 7357 L 7584 L 7824 ...,0,109,V1,256.708652,11.87702,"[7339, 7566, 7806, 8050, 8275, 8496, 8720, 895...",0 7200 0.662301 7201 0.62378...,False,23.637223
3,240.500000,16.702224,64,symbol 10845 L 11075 L 11310 ...,0,109,V1,256.708652,11.87702,"[10829, 11058, 11299, 11531, 11767, 12013, 122...",0 10800 0.299100 10801 0.27815...,True,16.208652
4,245.714286,7.823304,64,symbol 14463 L 14714 L 14958 ...,0,109,V1,256.708652,11.87702,"[14445, 14696, 14941, 15185, 15420, 15653, 159...",0 14400 0.463421 14401 0.43304...,False,10.994366


In [7]:
testset.head()

,R_interval_mean,R_interval_sd,age,annotation,gender,id,lead,p2p_mean,p2p_std,peak,strip,stripPVC,p2p_diff_form_norm
0,249.714286,33.990395,73,symbol 67 + 197 N 459 ...,1,105,V1,245.874749,42.059491,"[175, 445, 687, 944, 1201, 1466, 1728, 2000, 2...",0 0 0.407442 1 0.446399 2...,False,3.839537
1,263.416667,42.969870,73,symbol 3835 N 4102 N 4371 ...,1,105,V1,245.874749,42.059491,"[3814, 4083, 4358, 4614, 4879, 5134, 5387, 556...",0 3600 0.112379 3601 0.046940 3...,True,17.541918
2,261.461538,42.501793,73,symbol 7246 N 7506 N 7767 ...,1,105,V1,245.874749,42.059491,"[7225, 7518, 7746, 7925, 8270, 8551, 8808, 907...",0 7200 -0.063671 7201 -0.11752...,True,15.586789
3,257.307692,37.142935,73,symbol 10908 N 11168 N 11424 ...,1,105,V1,245.874749,42.059491,"[10886, 11156, 11412, 11655, 11917, 12166, 124...",0 10800 0.002881 10801 -0.00406...,True,11.432943
4,261.461538,7.131894,73,symbol 14517 N 14786 N 15053 ...,1,105,V1,245.874749,42.059491,"[14495, 14799, 15033, 15300, 15553, 15814, 160...",0 14400 -0.015548 14401 -0.03361...,False,15.586789


### Normalization

In [9]:
from sklearn import preprocessing

col = trainset.drop(['peak','annotation','strip','stripPVC','id','lead'],axis=1).columns.values
scale = preprocessing.StandardScaler()
scale.fit(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1))
n_trainset = pd.DataFrame(scale.transform(trainset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)
n_testset = pd.DataFrame(scale.transform(testset.drop(['peak','annotation','strip','stripPVC','id','lead'], axis=1)),columns=col)

### Calculate Baseline

In [8]:
len(trainset[trainset.stripPVC==True])/len(trainset)

0.568086592178771

### Build models

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
def metric(pred, real):
    ConfusionMatrix= pd.DataFrame(confusion_matrix(pred,real), columns=['True','False'], index=['True','False'])
    print(ConfusionMatrix)
    print('----------------')
    print('Precision:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0]))
    print('Recall:', ConfusionMatrix.iloc[0,0] / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]))
    print('Accuracy:', (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,1]) / (ConfusionMatrix.iloc[0,0] + ConfusionMatrix.iloc[1,0] + ConfusionMatrix.iloc[0,1] + ConfusionMatrix.iloc[1,1]))
    print('AUC:', roc_auc_score(pred,real))

#### Random forest

In [17]:
for i in range(3,7):
    clf = RandomForestClassifier(n_estimators=100, max_depth=3,max_features=i, random_state=0)
    clf.fit(n_trainset , trainset['stripPVC'])
    pred = clf.predict(n_testset)
    print('max features: ' , i)
    print(metric(pred,testset['stripPVC']))

max features:  3
       True  False
True    635      8
False   302    666
----------------
Precision: 0.6776947705442903
Recall: 0.4880860876249039
Accuracy: 0.8075729360645562
AUC: 0.8377874246494352
None
max features:  4
       True  False
True    631      7
False   306    667
----------------
Precision: 0.6734258271077909
Recall: 0.4861325115562404
Accuracy: 0.8057107386716326
AUC: 0.837268474517296
None
max features:  5
       True  False
True    627      8
False   310    666
----------------
Precision: 0.6691568836712913
Recall: 0.48491879350348027
Accuracy: 0.8026070763500931
AUC: 0.8348893119917387
None
max features:  6
       True  False
True    626      8
False   311    666
----------------
Precision: 0.6680896478121665
Recall: 0.4845201238390093
Accuracy: 0.8019863438857853
AUC: 0.8345301557268273
None



#### SVM

In [19]:
clf_svm = SVC(kernel='rbf')
clf_svm.fit(n_trainset, trainset['stripPVC'])
svm_pred = clf_svm.predict(n_testset)

In [20]:
metric(svm_pred ,testset['stripPVC'])

       True  False
True    655     69
False   282    605
----------------
Precision: 0.6990394877267876
Recall: 0.5198412698412699
Accuracy: 0.7821229050279329
AUC: 0.7933852703569672
